#More Efficient Weight Initialization

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
# parameters
learning_rate = 1e-3
training_epochs = 15
batch_size = 100

In [ ]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 120323416.99it/s]


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 14350870.02it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 32358075.13it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 20572925.24it/s]


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [ ]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
# nn layers
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [ ]:
# Xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-0.0469,  0.0613,  0.1208,  ..., -0.0088, -0.1442,  0.0517],
        [ 0.0614, -0.0203,  0.1185,  ...,  0.0404,  0.0880, -0.0122],
        [ 0.0959, -0.1310, -0.0171,  ..., -0.0223, -0.1097, -0.1196],
        ...,
        [-0.0758, -0.0502,  0.1196,  ...,  0.1320, -0.0851,  0.1238],
        [-0.1148,  0.0104,  0.0216,  ...,  0.1441,  0.0249, -0.1332],
        [-0.1414, -0.0291,  0.0772,  ...,  0.0031,  0.1308, -0.1378]],
       requires_grad=True)

In [ ]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [ ]:
# define cost / loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)
for epoch in range(training_epochs + 1):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.view(-1, 28 * 28).to(device)
    Y = Y.to(device)

    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost+= cost / total_batch

  print('Epoch {:6d}/{} Cost: {:.4f}'.format(
      epoch, training_epochs, avg_cost
  ))
print('learning finished')
# Xavier initialization 을 통한 weight initialization 에 의해 훨씬 낮은 cost 를 보이고 있다.

Epoch      0/15 Cost: 0.2145
Epoch      1/15 Cost: 0.0897
Epoch      2/15 Cost: 0.0626
Epoch      3/15 Cost: 0.0495
Epoch      4/15 Cost: 0.0403
Epoch      5/15 Cost: 0.0363
Epoch      6/15 Cost: 0.0296
Epoch      7/15 Cost: 0.0249
Epoch      8/15 Cost: 0.0242
Epoch      9/15 Cost: 0.0208
Epoch     10/15 Cost: 0.0200
Epoch     11/15 Cost: 0.0191
Epoch     12/15 Cost: 0.0172
Epoch     13/15 Cost: 0.0158
Epoch     14/15 Cost: 0.0187
Epoch     15/15 Cost: 0.0125
learning finished


In [ ]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9963499903678894
Label:  4
Prediction:  4


#Deep layers

In [ ]:
# nn layers
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU()

In [ ]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[ 0.0048, -0.0986, -0.0872,  ...,  0.0694, -0.0659, -0.0166],
        [ 0.0698,  0.0507,  0.1058,  ...,  0.0029,  0.0679, -0.0124],
        [-0.0656,  0.0859, -0.0042,  ..., -0.0868,  0.0171, -0.0910],
        ...,
        [ 0.0520,  0.0508, -0.0047,  ...,  0.0639,  0.0873,  0.0493],
        [-0.0615,  0.0896, -0.1004,  ..., -0.0363,  0.0605, -0.0627],
        [-0.0236, -0.0572,  0.0813,  ..., -0.0742, -0.0493, -0.0203]],
       requires_grad=True)

In [ ]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3, relu, linear4, relu, linear5).to(device)